In [3]:
import os
import json
import torch

from utils import find_assistant_token_end
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model = AutoModelForCausalLM.from_pretrained("/nlsasfs/home/isea/isea10/aansh/deception_detection/weights/Llama-3.1-8B-Instruct", device_map="auto", dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("/nlsasfs/home/isea/isea10/aansh/deception_detection/weights/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
with open("/nlsasfs/home/isea/isea10/aansh/introspection/data/exp1/multi_token_completions/simple_narrative_continuations_category_1.json", "r") as f:
    data = json.load(f)
    
example = data[0]

In [6]:
user_prompt = f"""
Complete the following sentence:
{example["prompt"]} 
"""
message_1 = [
    {"role": "user", "content": user_prompt}, 
    {"role": "assistant", "content": example["completion_1"]},
]
inputs_temp_1 = tokenizer.apply_chat_template(message_1, tokenize=False)
inputs_1 = tokenizer(inputs_temp_1, return_tensors="pt").to(model.device)
detect_token_idx_1 = find_assistant_token_end(inputs_1["input_ids"][0].tolist())

In [7]:
inputs_temp_1

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nComplete the following sentence:\nShe opened the door<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nand saw her friend waiting on the porch.<|eot_id|>'

In [15]:
inputs_1["input_ids"].shape

torch.Size([1, 55])

In [9]:
detect_token_idx_1

44

In [16]:
tokenizer.decode(inputs_1["input_ids"][0][-1])

'<|eot_id|>'

In [13]:
outputs = model(**inputs_1)
logits = outputs.logits

In [17]:
tokenizer.decode(logits[0, -1].argmax().item())

'<|start_header_id|>'